## Alerte à la poste 

#### 1. Parcourez le fichier pour trouver la liste de tous les utilisateurs qui se sont connectés, enregistrez cette liste dans un fichier utilisateurs.txt. 

In [24]:
file1 = "connexion.log"
users = []
users_not_uni = []

with open(file1, "r") as logs :
    for line in logs:
        mocs = line.split(';')
        login = mocs[1]
        users_not_uni.append(login)
        if login not in users :
            users.append(login)
        
ready_file_path = "utilisateurs.txt"
with open(ready_file_path,'w') as users_file:
    for user in users :
        users_file.write(f"{user}\n")

        
print("Number of all connections : ", len(users_not_uni))
print("Number of users is : ",len(users))
users[:10] 

Number of all connections :  3488
Number of users is :  3373


['maddux',
 'nose',
 'waterline',
 'worlock',
 'pass',
 'disentangle',
 'cursory',
 'luxurious',
 'assurance',
 'krause']

#### 2. Les personnes se sont connectées en dehors des heures d'ouverture du bureau

In [22]:
import pandas as pd 
# Here Pandas is only for my pleasure in visualizing the final results

file1 = "connexion.log"
out_of_working_hours = "dangerous_connections.txt"

def closed_office_time(time_str):
    time_parts = time_str.split(' ')[1]
    hour = int(time_parts.split(':')[0])
    return hour<8 or hour>=19

closed_office_con_data = []

with open(file1, "r" ) as logs, open(out_of_working_hours, "w") as out:
    for line in logs:
        mocs = line.split(';')
        ip = mocs[0]
        login = mocs[1]
        time_str = mocs[2].strip()
        
        if closed_office_time(time_str):
            out.write(f"{ip};{login};{time_str}\n")
            closed_office_con_data.append([ip, login, time_str])

print("How many connections were there when the office was closed :",  len(closed_office_con_data))                      
df = pd.DataFrame(closed_office_con_data, columns=['IP','Login','Timestamp'])
# Let's look for example of dataframe
df.head(10)


How many connections were there when the office was closed : 50


,IP,Login,Timestamp
0,93.170.131.108,pisistratus,26/04/17 00:37
1,93.170.131.108,pisistratus,19/09/19 19:32
2,93.170.131.108,pisistratus,09/02/18 19:41
3,93.170.131.108,pisistratus,13/11/19 00:24
4,93.170.131.108,pisistratus,17/04/16 23:51
5,93.170.131.108,pisistratus,22/04/15 00:29
6,93.170.131.108,pisistratus,10/08/18 22:03
7,93.170.131.108,pisistratus,14/05/17 22:24
8,93.170.131.108,pisistratus,18/01/19 00:10
9,93.170.131.108,pisistratus,22/11/15 19:17


In [13]:
# We should know how many connections were when the office was closed 
df.shape

(50, 3)

#### 3. on produira un fichier suspect.txt avec une ligne par utilisateur et le nombre de fois qu'il s'est connecté à une ip interdite

In [17]:
warning_file = "warning.txt"

red_ips = []
with open(warning_file , 'r') as ips :
    for line in ips:
        red_ips.append(line.strip())
        
red_ips

['92.63.87.106',
 '92.63.87.134',
 '92.63.87.48',
 '92.63.87.53',
 '93.170.104.127',
 '93.170.123.119',
 '93.170.123.185',
 '93.170.123.60',
 '93.170.131.108',
 '93.170.169.52',
 '94.242.57.45']

In [20]:
suspect_file = "suspect_file.txt"

user_ip_count = {}
for entry in closed_office_con_data:
    ip, login, time_str = entry
    if ip in red_ips:
        if login not in user_ip_count:
            user_ip_count[login] = 0
        user_ip_count[login] += 1

with open(suspect_file, 'w') as suspect_out:
    for user, count in user_ip_count.items():
        suspect_out.write(f"{user} {count}\n")

#### Version optimisée

In [27]:
file1 = "connexion.log"
ready_file_path = "utilisateurs.txt"
out_of_working_hours = "dangerous_connections.txt"
warning_file = "warning.txt"
suspect_file = "suspect_file.txt"

users = []
with open(file1, "r") as logs :
    for line in logs:
        mocs = line.split(';')
        login = mocs[1]
        if login not in users :
            users.append(login)
            
with open(ready_file_path,'w') as users_file:
    for user in users :
        users_file.write(f"{user}\n")

def closed_office_time(time_str):
    time_parts = time_str.split(' ')[1]
    hour = int(time_parts.split(':')[0])
    return hour<8 or hour>=19

closed_office_con_data = []
with open(file1, "r" ) as logs, open(out_of_working_hours, "w") as out:
    for line in logs:
        mocs = line.split(';')
        ip = mocs[0]
        login = mocs[1]
        time_str = mocs[2].strip()
        
        if closed_office_time(time_str):
            out.write(f"{ip};{login};{time_str}\n")
            closed_office_con_data.append([ip, login, time_str])
            
red_ips = []
with open(warning_file , 'r') as ips :
    for line in ips:
        red_ips.append(line.strip())       
        
user_ip_count = {}
for entry in closed_office_con_data:
    ip, login, time_str = entry
    if ip in red_ips:
        if login not in user_ip_count:
            user_ip_count[login] = 0
        user_ip_count[login] += 1

with open(suspect_file, 'w') as suspect_out:
    for user, count in user_ip_count.items():
        suspect_out.write(f"{user} {count}\n")

print("User who use IPs from the list of warnings when the office is closedd => ", user_ip_count)

User who use IPs from the list of warnings when the office is closedd =>  {'pisistratus': 46}
